# Power Grid Stability Demo
### Group member:
Liam Dietrich
Andy Nguyen
Alex Semenov
Benjamin Rockman

Imports necessary for the demo.

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from time import sleep
from IPython.display import clear_output

This is the base power grid agent that includes 2 attributes that the city and generator agent also inherit, name and the graph.

In [ ]:
class PowerGridAgent:
    def __init__(self, name, graph):
        self.name = name
        self.graph = graph
    
    def get_name(self):
        return self.name
    
    def get_graph(self):
        return self.graph

## Generator Agent Class

GeneratorAgent is the base class for the generator agent. This agent includes important attributes such as power_output, capacity and status. It also has necessary behaviour such as get_remaining_capacity() or generate_power().

In [ ]:
class GeneratorAgent(PowerGridAgent):
    def __init__(self, name, capacity, power_output, p_off, graph):
        super().__init__(name, graph)
        self.capacity = capacity
        self.power_output = power_output
        self.status = True
        self.p_off = p_off

    def generate_power(self, demand):
        if not self.status:
            return 0
        return min(self.power_output, self.capacity, demand)
    
    def get_capacity(self):
        return self.capacity
    
    def get_remaining_capacity(self):
        if not self.status:
            return 0
        return self.capacity - self.power_output
    
    def is_on(self):
        return self.status
    
    def start(self):
        self.status = True

    def stop(self):
        self.status = False

    def turn_off_generator(self):
        if random.random() < self.p_off:
            self.status = False

## City Agent Class

Similarly, an agent class name CityAgent, also inherit from PowerGridAgent. This agent class is a bit more sophisticated with 2 of the most important behaviours. The first is consume_power(), this method simply draws the power from the generator until there is nothing left in the generator or when the demand of that particular city is fulfilled. The other method, which does not work correctly (but efforts were made), but works when only one city demands energy from others and does not work when there are multiple cities demanding "help" from neighbouring cities

In [ ]:
class CityAgent(PowerGridAgent):
    def __init__(self, city_name, demand=0, consumption=0, graph=None, grid=None):
        super().__init__(city_name, graph)
        self.city_name = city_name
        self.demand = demand
        self.consumption = consumption
        self.sharing = 0
        self.share_taking = 0
        self.grid = grid

    def update_demand(self, demand):
        self.demand = demand

    def update_consumption(self, consumption):
        self.consumption = consumption

    def update_sharing(self, share):
        self.sharing = share
    
    def update_share_taking(self, share_take):
        self.share_taking = share_take

    def get_consumption(self):
        return self.consumption

    def get_demand(self):
        return self.demand
    
    def get_sharing(self):
        return self.sharing
    
    def get_share_taking(self):
        return self.share_taking

    def consume_power(self):
        # Calculate how much power the city needs
        demand = self.demand
        consumption = self.consumption
        power_drawn = 0
        # Find all the generator nodes that are connected to the city node
        generators = [n for n in self.graph.neighbors(self.get_name())
                      if isinstance(self.graph.nodes[n]['agent'], GeneratorAgent)]

        # Iterate over the generators and try to draw power from each one
        for generator in generators:
            # Check if the generator is turned on and has remaining capacity
            if self.graph.nodes[generator]['agent'].is_on():
                if consumption < demand:
                    if self.graph.nodes[generator]['agent'].get_remaining_capacity() > 0:
                        diff = demand-consumption
                        # Calculate the maximum amount of power that can be drawn from the generator
                        max_power = self.graph.nodes[generator]['agent'].get_remaining_capacity()

                        # Draw power from the generator
                        power_drawn = min(max_power, diff)

                        # Update the remaining demand
                        consumption += power_drawn
                        self.consumption = consumption
                        self.graph.nodes[generator]['agent'].power_output += power_drawn
                        demand -= consumption
                elif demand < consumption:
                    diff = consumption-demand
                    power_drawn = demand
                    consumption = power_drawn
                    self.consumption = consumption
                    self.graph.nodes[generator]['agent'].power_output -= diff
                    demand -= consumption
        return power_drawn
    
    def consume_from_neighbours(self):
        demand = self.demand
        consumption = self.consumption
        cum_power = 0
        
        neighbours = [n for n in self.graph.neighbors(self.get_name())
                      if isinstance(self.graph.nodes[n]['agent'], CityAgent)]
        
        diff = demand-consumption
        for neighbour in neighbours:
            old_demand = self.graph.nodes[neighbour]['agent'].demand
            self.graph.nodes[neighbour]['agent'].update_demand(diff + self.graph.nodes[neighbour]['agent'].demand) #Update demand of neighbour to include current node demand
            power = self.graph.nodes[neighbour]['agent'].consume_power()
            #self.grid_edges[self.get_name(), self.graph.nodes[neighbour]['agent'].get_name()] = power #Change edge label to reflect city-city power draw
            sharing = self.graph.nodes[neighbour]['agent'].consumption-old_demand
            self.graph.nodes[neighbour]['agent'].sharing = sharing
            self.share_taking = sharing
            cum_power += sharing
            diff -= sharing
            if diff == 0: #Break if difference is drawn, no need to draw more from other neighbours
                break
        if diff > 0:
            if not self.grid.is_generators_maxed():
                for neighbour in neighbours:
                    borrowed_power = self.graph.nodes[neighbour]['agent'].consume_from_neighbours()
                    self.graph.nodes[neighbour]['agent'].sharing += borrowed_power
                    self.share_taking += borrowed_power

            else:
                cum_power = -1
        return cum_power


## Power Grid Class (Main class)

This class is where everything is implemented. Some of the important features this class has include initializing the graph with appropriate amount of generators and cities, connect the adjacent cities and generators only, create generator/city node based on the name and link it with the node in the graph. Another particularly important method is the update_power_grid() method where the step function appears. This method changes the demand of the cities with a sine wave and a randomize (controlled) demand, then it draws power from the generator and neighbouring nodes if it doesn't have enough. 

In [ ]:
class PowerGrid:
    def __init__(self, numGenerator):
        self.numGenerator = numGenerator
        self.numCity = numGenerator * 2

        self.position = {}
        self.grid_edges = {}
        self.generator_agents = {}
        self.city_agents = {}

        self.graph = nx.Graph()

        # Add generator nodes to the graph and position them on the left, spaced out evenly
        for i in range(1, self.numGenerator + 1):
            genName = f'Gen{i}'
            self.position[genName] = (-90 + i*40, 100)
            if i > 1:
                prevGenName = f'Gen{i-1}'
                self.grid_edges[(prevGenName, genName)] = 0

        # Add city nodes to the graph and position them on the right, spaced out evenly
        for i in range(1, self.numCity + 1):
            cityName = f'City{i}'
            self.position[cityName] = (-100 + i*30, 35)
            genIndex = int((i - 1) / 2) + 1
            genName = f'Gen{genIndex}'
            self.grid_edges[(genName, cityName)] = 0

            if i > 1:
                prevCityName = f'City{i-1}'
                self.grid_edges[(prevCityName, cityName)] = 0

            if i < self.numCity:
                nextCityName = f'City{i+1}'
                self.grid_edges[(cityName, nextCityName)] = 0

        self.graph.add_nodes_from(self.position.keys())
        self.graph.add_edges_from(self.grid_edges)

    def add_generator_agent(self, name, capacity, power_output=0, p_off=0):
        # Create a new GeneratorAgent and add it to a generator node in the graph
        # node_name = f'Gen{len(self.graph.nodes) - self.numCity}'
        agent = GeneratorAgent(
            name, capacity, power_output, p_off, graph=self.graph)
        self.graph.nodes[name]['agent'] = agent
        self.generator_agents[name] = agent

    def add_city_agent(self, name, demand=0, consumption=0):
        # Create a new CityAgent and add it to a city node in the graph
        # node_name = f'City{city_id}'
        agent = CityAgent(name, demand, consumption,
                          graph=self.graph, grid=self)
        self.graph.nodes[name]['agent'] = agent
        self.city_agents[name] = agent

    def get_city_agent(self, city_name):
        if isinstance(self.graph.nodes[city_name]['agent'], CityAgent):
            return self.graph.nodes[city_name]['agent']
        else:
            return None

    def get_generator_agent(self, city_name):
        if isinstance(self.graph.nodes[city_name]['agent'], GeneratorAgent):
            return self.graph.nodes[city_name]['agent']
        else:
            return None

    def draw(self, time_step=0):
        # Draw the graph
        nx.draw(self.graph, self.position,
                node_color='lightgreen',
                node_shape='s',
                node_size=2500,
                with_labels=False)
        plt.title(f"Time step: {time_step}") 

        # Add labels to the nodes
        labels = {}
        for node in self.graph.nodes:
            agent = self.graph.nodes[node]['agent']
            if isinstance(agent, GeneratorAgent):
                labels[node] = f"{node}\nCapacity: {int(agent.get_capacity())}\nRemain: {int(agent.get_remaining_capacity())}"
            elif isinstance(agent, CityAgent):
                labels[node] = f"{node}\n\nDemand: {int(agent.get_demand()) if np.isfinite(agent.demand) else 'inf'}"

        nx.draw_networkx_labels(self.graph, self.position,
                            labels=labels, font_size=7)

        # Add labels to the edges
        nx.draw_networkx_edge_labels(
            self.graph, self.position, edge_labels=self.grid_edges)

        plt.axis('equal')
        plt.show()

    def update_edge_value(self):
        # Update the edges between generators and cities with power consumption from the city
        for edge in self.grid_edges:
            start_node, end_node = edge
            if isinstance(self.graph.nodes[start_node]['agent'], CityAgent) and isinstance(self.graph.nodes[end_node]['agent'], GeneratorAgent):
                generator_agent = self.graph.nodes[start_node]['agent']
                consumption = generator_agent.get_consumption()
                self.grid_edges[edge] = int(consumption)
            elif isinstance(self.graph.nodes[end_node]['agent'], CityAgent) and isinstance(self.graph.nodes[start_node]['agent'], GeneratorAgent):
                generator_agent = self.graph.nodes[end_node]['agent']
                consumption = generator_agent.get_consumption()
                self.grid_edges[edge] = int(consumption)
            elif isinstance(self.graph.nodes[end_node]['agent'], CityAgent) and isinstance(self.graph.nodes[start_node]['agent'], CityAgent):
                generator_agent = self.graph.nodes[end_node]['agent']
                sharing = generator_agent.sharing
                if sharing == float('inf') or sharing == float('-inf'):
                    self.grid_edges[edge] = -1
                else:
                    self.grid_edges[edge] = int(sharing)
            else:
                self.grid_edges[edge] = 0

    def generate_demand_changes(self, amplitude, period, phase, num_cities, time_step):
        demand_changes = {}
        time = time_step
        for i in range(num_cities):
            # use a sine wave to generate demand changes
            demand_change = abs(amplitude * random.uniform(0.75, 1.5) * math.sin(2 * math.pi / period * time + phase))
            demand_changes[f"City{i+1}"] = demand_change
        return demand_changes

    def is_generators_maxed(self):
        for generator in self.generator_agents:
            if self.generator_agents[generator].get_remaining_capacity() != 0:
                return False
        return True

    def update_power_grid(self, demand=500, steps=1, draw=False):
        blackouts = 0
        brownouts = 0
        totBlackouts = 0
        totBrownouts = 0

        for time_step in range(0, steps):

            # Generate demand changes for each city agent
            demand_changes = self.generate_demand_changes(amplitude=demand, period=10, phase=0, num_cities=self.numCity, time_step=time_step)

            # Update the demand for each city agent
            for city_name, demand_change in demand_changes.items():
                city_agent = self.get_city_agent(city_name)
                if city_agent:
                    city_agent.update_demand(int(demand_change))

            # Consume power for each city agent
            for city_node in self.graph.nodes:
                if isinstance(self.graph.nodes[city_node]['agent'], CityAgent):
                    city_agent = self.graph.nodes[city_node]['agent']
                    city_agent.consume_power()

            for city_node in self.graph.nodes:
                if isinstance(self.graph.nodes[city_node]['agent'], CityAgent):
                    city_agent = self.graph.nodes[city_node]['agent']
                    if city_agent.consumption < city_agent.demand:
                        city_agent.consume_from_neighbours()

            # Check the status of the city
            for node in self.graph.nodes:
                if isinstance(self.graph.nodes[node]['agent'], CityAgent):
                    city_agent = self.graph.nodes[node]['agent']
                    if city_agent.consumption < 0.4 * city_agent.demand:
                        blackouts += 1
                    elif city_agent.consumption >= 0.4 * city_agent.demand and city_agent.consumption < city_agent.demand:
                        brownouts += 1
                    
            # Update the edge values with power consumption from the cities and redraw the graph
            self.update_edge_value()
            
            if draw:
                # Redraw the graph
                self.draw(time_step=time_step)
            
        totBlackouts = int(blackouts/self.numCity)
        totBrownouts = int(brownouts/self.numCity)
        
        return (totBlackouts, totBrownouts)

## Demo 1 (single time step)

The first demo demonstrate the acquisition of the average value of average probability of blackout and brownout given a ratio of capacity to demand of the power grid (resiliency) at one time step through running the stepper function for 10 time steps (computer time)

In [ ]:
# Example of a step of running through for a specific ratio demand and supply

power_ratio = 1.0
generate = 10000
demand = 5000
steps = 10

# Create a power grid with 3 generators
power_grid = PowerGrid(numGenerator=2)

# Add generator agents to the power grid
power_grid.add_generator_agent(name='Gen1', capacity=(power_ratio*generate), p_off=0.5)
power_grid.add_generator_agent(name='Gen2', capacity=(power_ratio*generate), p_off=0.5)

# Add city agents to the power grid
power_grid.add_city_agent('City1')
power_grid.add_city_agent('City2')
power_grid.add_city_agent('City3')
power_grid.add_city_agent('City4')

sumBlack = 0
sumBrown = 0
# Run the stepper function 10 times to get the average rate of black/brown out based on the power ratio
for i in range(0, 10):
    black, brown = power_grid.update_power_grid(demand=demand, steps=steps)
    sumBlack += (black/steps)
    sumBrown += (brown/steps)

avgBlack = round((sumBlack/10) * 100, 2)
avgBrown = round((sumBrown/10) * 100, 2)

print(f"Grid demand = {demand * 4}\n")
print(f"Grid capacity = {2 * (power_ratio*generate)}\n")
print(f"Supply/Demand ratio = {power_ratio}\n")
print(f"Average prob of blackout = {avgBlack} %\n")
print(f"Average prob of brownout = {avgBrown} %\n")

At a ratio of supply/demand of 1.0, the average probabilty of blackout is very low 0-2%, rarely 3%. But the probablity of brownout is relatively high 15-30%. Blackout is counted when the power consumption by the city is 40% lower than its demand. Brownout is counted when the power consumption is in the range of 40% to 85% of the demand. These number are arbitrary, but they somehow are good representation of the outages.

## Demo 2 (Gathering of info of different ratio)

So what happens at different ratios of capacity/demand? That is what going to be explore now as the ratio is fluctuate between 0 and 2 with a time step of 0.1, values higher than 2 are somewhat irrelevant as more double the size of the demand will drive much higher cost and might not reflect true capacity/demand ratio in real life. Furthermore a plot between average rate of blackout and brownout will be plot to find the critical point where the rate of blackout and brownouts drops the most. 

In [ ]:
# Gathering the list of average blackout and brownout rate to the capacity/demand ratio of the grid

# List of values range from 0-2 with step of 0.1
ratio_list = [i/10 for i in range(0, 21)]

# Default demand and capacity of the cities and grid
generate = 10000
demand = 5000
steps = 10

total_black_prob = []
total_brown_prob = []

for ratio in ratio_list:
    # Create a power grid with 3 generators
    power_grid = PowerGrid(numGenerator=2)

    # Add generator agents to the power grid
    power_grid.add_generator_agent(name='Gen1', capacity=(ratio*generate), p_off=0.5)
    power_grid.add_generator_agent(name='Gen2', capacity=(ratio*generate), p_off=0.5)

    # Add city agents to the power grid
    power_grid.add_city_agent('City1')
    power_grid.add_city_agent('City2')
    power_grid.add_city_agent('City3')
    power_grid.add_city_agent('City4')

    sumBlack = 0
    sumBrown = 0
    for i in range(0, 10):
        black, brown = power_grid.update_power_grid(demand=demand, steps=steps)
        sumBlack += (black/steps)
        sumBrown += (brown/steps)

    avgBlack = round((sumBlack/10) * 100, 2)
    avgBrown = round((sumBrown/10) * 100, 2)

    total_black_prob.append(avgBlack)
    total_brown_prob.append(avgBrown)

The total amount of blackouts and brownouts at every ratio is now plotted.

Blackout to ratio plot:

In [ ]:
plt.plot(ratio_list, total_black_prob)
plt.title("Capacity/demand ratio to probability of black out happens in a grid")
plt.xlabel("Capacity/Demand ratio in the grid")
plt.ylabel("Average probability of blackout (%)")

Brownout to ratio plot:

In [ ]:
plt.plot(ratio_list, total_brown_prob)
plt.title("Capacity/demand ratio to probability of brownout happens in a grid")
plt.xlabel("Capacity/Demand ratio in the grid")
plt.ylabel("Average probability of brownout (%)")

## Demo 3 (Visual representation)

Unfortunatly, the team was not able to get the consume_from_neighbours() function works correctly as it is complex and very intricate, involving multiple layers. Under the condition of only one node need help from its neighbour, the method works fine. However, when there are more than one city node demand help, its impact is too large and eventually breaks the simulation. However, an attempt at making the simulation to be "single-source" was made in a separate source file simulating the implementation of a single point source microgrid and how it distribute power effectively within its nodes. That said, by running the code below, the graphs will be drawn and demonstrates the current work of the team.

In [ ]:
ratio = 1.0
generate = 1000
demand = 500
steps = 10
# Create a power grid with 3 generators
power_grid = PowerGrid(numGenerator=2)

# Add generator agents to the power grid
power_grid.add_generator_agent(name='Gen1', capacity=(ratio*generate), p_off=0.5)
power_grid.add_generator_agent(name='Gen2', capacity=(ratio*generate), p_off=0.5)

# Add city agents to the power grid
power_grid.add_city_agent('City1')
power_grid.add_city_agent('City2')
power_grid.add_city_agent('City3')
power_grid.add_city_agent('City4')

power_grid.update_power_grid(demand=500, steps=10, draw=True)